In [21]:
import pandas as pd
import numpy as np
from util import make_adjust, make_overheat

In [22]:
df = pd.read_csv('data/region_inter.csv')

In [23]:
# 조정대상지역, 투기과열지구 추가
df = make_overheat(make_adjust(df))

In [24]:
# 이상치 제거
to_drop_index = [39540, 30849, 2601, 26965]
df = df.drop(index=to_drop_index)

In [25]:
# 광역행 만들기
df['광역'] = df['시'].str.split().str[0]

# 줄이기 위한 광역 키
REGIONKEY = {'강원특별자치도': '강원', '경기도': '경기', '경상남도': '경남',
            '경상북도': '경북', '광주광역시': '광주', '대구광역시': '대구', '대전광역시':'대전',
            '부산광역시': '부산', '서울특별시': '서울', '세종특별자치시': '세종', '울산광역시':'울산',
            '인천광역시': '인천', '전라남도':'전남', '전라북도':'전북', '제주특별자치도':'제주', '충청남도':'충남', '충청북도':'충북'}
df['광역'] = df['광역'].replace(REGIONKEY)

In [26]:
# 로그 변환
df['log_면적당보증금'] = np.log(df['면적당보증금'])
df['log_면적당매매금'] = np.log(df['면적당매매금'])

In [27]:
# 전세가율 행 추가
df['전세가율'] = df['면적당보증금'] / df['면적당매매금']

# Target 추가, 전세가율(60% 이상, 70%,..., 90%) 각각 target60, target70,..., target90으로 하고 이상이면 1, 이하면 0
df['target60'] = (df['전세가율'] >= 0.6).astype(int)
df['target70'] = (df['전세가율'] >= 0.7).astype(int)
df['target80'] = (df['전세가율'] >= 0.8).astype(int)
df['target90'] = (df['전세가율'] >= 0.9).astype(int)

In [28]:
# 학습에 사용할 특성 선별
need_features = ['시군구', '계약년월', '면적당보증금', '조정대상지역', '투기과열지구', '광역', 'target60', 'target70', 'target80', 'target90']
df = df.loc[:,need_features]

In [29]:
# 데이터셋 최종확인
df

,시군구,계약년월,면적당보증금,조정대상지역,투기과열지구,광역,target60,target70,target80,target90
0,강원특별자치도 강릉시 견소동,201908,207.164427,0,0,강원,1,1,1,1
1,강원특별자치도 강릉시 견소동,201909,199.646793,0,0,강원,1,1,1,0
2,강원특별자치도 강릉시 견소동,201910,222.682534,0,0,강원,1,1,1,0
3,강원특별자치도 강릉시 견소동,201911,260.329239,0,0,강원,1,1,1,1
4,강원특별자치도 강릉시 견소동,201912,220.911840,0,0,강원,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
96427,충청북도 충주시 호암동,202303,259.748017,0,0,충북,1,1,1,1
96428,충청북도 충주시 호암동,202304,190.967701,0,0,충북,1,1,0,0
96429,충청북도 충주시 호암동,202305,237.558371,0,0,충북,1,1,1,0
96430,충청북도 충주시 호암동,202306,315.277732,0,0,충북,1,1,1,1


In [20]:
df.to_csv('data/dataset.csv', index=False)